### spatialy explicit growth curve by region
This notebook assumes you have a json of growth parameters, if you don't please generate them using carbon_loop.ipynb.

To create spatialy explicit growth curves two geographic boundarys need to be combined:
1. continental region
2. climatic region (gez in this example)

Climatic regions are grouped into dry and humid climates. Then continental regions and climatic regions are converted to raster images and paired using Cantor paring. 

In [ ]:
import ee
import geemap
from cantor import *


ee.Initialize()
gez = ee.FeatureCollection("projects/sig-misc-ee/assets/sbp/ecofloristic_zones")
CONTINENTAL_REGIONS = ee.FeatureCollection("projects/sig-misc-ee/assets/sbp/continental_regions")

In [ ]:
def carbon_growth_function(stand_age:ee.Image, paramters_stack:ee.Image):
  #// b0(1-EXP(-b2Age))^b2
  carbonHa = ee.Image(0).expression("(b0 *( 1 - exp)) ** b2",{
    'b0': paramters_stack.select('b0'),
    'exp': (stand_age.multiply(paramters_stack.select('b1').multiply(-1))).exp(),
    'b2': ee.Image(paramters_stack.select('b2'))
  }).rename('carbon');
  
  return carbonHa


def get_areas(image, geometry, scale=100):
    
    image = image.rename("image").round()
    pixelArea = ee.Image.pixelArea().divide(10000)
    reducer = ee.Reducer.sum().group(1, "image")

    areas = (
        pixelArea.addBands(image)
        .reduceRegion(reducer=reducer, geometry=geometry, scale=100, maxPixels=1e12)
        .get("groups")
    )

    areas_list = ee.List(areas).map(lambda i: ee.Dictionary(i).get("sum"))

    total = areas_list.reduce(ee.Reducer.sum())

    sum_img = image.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geometry,
        scale=100,
        maxPixels=1e13,
    )

    return areas, total


In [ ]:
# gez.aggregate_array('GEZ_TERM').distinct().getInfo()

humid_gez = [
    'Tropical rainforest',
    'Tropical moist deciduous forest'
    'Temperate oceanic forest',
    'Temperate continental forest',
    'Boreal coniferous forest',
    'Subtropical humid forest',
]
dry_gez = [
    'Temperate steppe',
    'Subtropical desert',
    'Subtropical steppe',
    'Tropical shrubland',
    'Tropical dry forest',
    'Tropical desert',
    'Subtropical dry forest',
    'Temperate desert',
    'Boreal tundra woodland',
]

excluded = [ 
    'Water',
    'No data',
    'Polar',
]
unknown = [
    'Boreal mountain system',
    'Tropical mountain system',
    'Subtropical mountain system',
    'Temperate mountain system',
]

In [ ]:
gez_map = ee.Dictionary({
    #1 humid
    'Tropical rainforest':1,
    'Tropical moist deciduous forest':1,
    'Temperate oceanic forest':1,
    'Temperate continental forest':1,
    'Boreal coniferous forest':1,
    'Subtropical humid forest':1,
    #2dry
    'Temperate steppe':2,
    'Subtropical desert':2,
    'Subtropical steppe':2,
    'Tropical shrubland':2,
    'Tropical dry forest':2,
    'Tropical desert':2,
    'Subtropical dry forest':2,
    'Temperate desert':2,
    'Boreal tundra woodland':2,
    #3/4 nd
    'Water':3,
    'No data':3,
    'Polar':3,
    'Boreal mountain system':4,
    'Tropical mountain system':4,
    'Subtropical mountain system':4,
    'Temperate mountain system':4,
})


In [ ]:

continental_map = ee.Dictionary({
    'Asia':5,
    'Asia (insular)':5,
    'Indian Ocean':5,
    'Australia':5,
    'New Zealand':5,
    
    'Africa':4,
    'Europe':3,
    'South America':2,
    'North America':1,
    

    
    'Pacific Ocean':6,
    'Atlantic Ocean':6,
    'Antarctica':6,
    'Arctic Ocean':6,
})


In [ ]:
# set a property 'climate_code' to each feature matching in the gez_map
gez_region_code = gez.map(lambda f: f.set('climate_code', gez_map.get(f.get('GEZ_TERM'))))

# set a property 'continential_code' to each feature matching in the continental_map
continential_region_code = CONTINENTAL_REGIONS.map(lambda f: f.set('continential_code', continental_map.get(f.get('REGION'))))
# print(continential_region_code.first().getInfo()['properties'])

# rasterize features for both continential_region_code and gez_region_code
empt = ee.Image()
gez_region_img = empt.paint(gez_region_code,'climate_code')
continential_region_img = empt.paint(continential_region_code,'continential_code')

# uniquly combine each image 
paired = eePair(gez_region_img, continential_region_img).int()

In [ ]:

Map = geemap.Map()
Map.addLayer(CONTINENTAL_REGIONS)
Map.addLayer(gez_region_img)
Map.addLayer(continential_region_img)
Map.addLayer(paired)
Map

 # manual work!
 Draw a rectangle on the map that covers the LMIC's. 
 

In [ ]:
# to identify each unique class' use the drawn area and stratify sample each class.
aoi = ee.FeatureCollection(Map.draw_last_feature)
samples = paired.int().stratifiedSample(numPoints=1, region=aoi, scale=1000)
print(samples.size().getInfo())                                 

Decode what each class represents. To do this we create a simplified dictionary of the new classes (1-6 for continential region, 1-2 for climatic region). The combined values are decoded and then joined back to the combined values for readability. 

# manual work!
Once you see the list of dry/humid and continental region, you will need to update `carbon.json`.

for each entry add a new key `code` and put the unique value of the combined map based on it's continent and climatic region. For instance, the code would be 33 for "Natural Asia Dry" since 
`(33, {'gez': 'dry', 'con': 'asia'}),`

You would update the json to look like:
```
"natural_asia_dry": {
    "name": "Natural Asia Dry",
    "parameters": [
        6.952180603672807,
        0.07708632118229979,
        2.155374937783795
    ],
    "code":33
}
```


In [ ]:
con_cs = {
    '5':'asia',
    '4':'africa',
    '3':'europe',
    '2':'south amrc',
    '1':'north amrc',
    '6':'na'
}
gez_cs = {
    '1':'humid',
    '2':'dry',
    # '3':'na',
    
}
uvalues = samples.aggregate_array('constant').distinct().getInfo()
dp = list(map(depair, uvalues))
# manually update "code" field in carbon.json based on values. 
dp = [{'gez':gez_cs.get(str(int(i[0])),'na'), 'con':con_cs.get(str(int(i[1])),'na')} for i in dp]
zipd = zip(uvalues,dp)
list(zipd)

In [ ]:
import json
# inputs, json path, paired img
path = r'carbon.json'
# paired
with open(path) as json_file:
    data = json.load(json_file)
data

# make images of coefs
# update base img where code
# area * value
out = ee.Image(0)
for k, v in data.items():
    print(v)
    coe1, coe2, coe3 = v['parameters']
    coe_stack = ee.Image.cat([ee.Image(coe1), ee.Image(coe2), ee.Image(coe3)]).select([0,1,2],['b0','b1','b2'])
    v['carbon_40y'] = carbon_growth_function(ee.Image(40), coe_stack)
    out = out.where(paired.eq(v['code']), v['carbon_40y'])



Map.addLayer(out.selfMask(),{},'final maps')
# todo export out, 100m